In [6]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install opentrons

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached opentrons-5.0.0-py2.py3-none-any.whl (1.1 MB)
  Using cached typing_extensions-3.10.0.0-py3-none-any.whl (26 kB)
  Using cached jsonschema-3.0.2-py2.py3-none-any.whl (54 kB)
  Using cached opentrons_shared_data-5.0.0-py2.py3-none-any.whl (434 kB)
  Using cached pydantic-1.8.2-py3-none-any.whl (126 kB)
  Using cached aionotify-0.2.0-py3-none-any.whl (6.6 kB)
  Using cached numpy-1.22.2-cp38-cp38-macosx_11_0_arm64.whl (12.7 MB)
  Using cached pyserial-3.5-py2.py3-none-any.whl (90 kB)
  Using cached anyio-3.3.0-py3-none-any.whl (77 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.1.1
    Uninstalling typing_extensions-4.1.1:
      Successfully uninstalled typing_extensions-4.1.1
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-

In [11]:
from opentrons import protocol_api
from opentrons import simulate

In [16]:
metadata = {
    'protocolName': 'Pipette Printer',
    'author': 'Alex Hadik <ahadik@mit.edu>',
    'description': 'A simple protocol template for the pipette printing lab.',
    'apiLevel': '2.11'
}

protocol = simulate.get_protocol_api('2.11')
p300 = protocol.load_instrument('p300_single', 'right')
tiprack = protocol.load_labware('opentrons_96_tiprack_300ul', '1')
palette = protocol.load_labware('usascientific_12_reservoir_22ml', '2')
canvas = protocol.load_labware('nest_96_wellplate_200ul_flat', '5')

/Users/alex/.opentrons/robot_settings.json not found. Loading defaults
/Users/alex/.opentrons/deck_calibration.json not found. Loading defaults


In [30]:
dir(palette)
canvas.rows_by_name()

{'A': [A1 of NEST 96 Well Plate 200 µL Flat on 5,
  A2 of NEST 96 Well Plate 200 µL Flat on 5,
  A3 of NEST 96 Well Plate 200 µL Flat on 5,
  A4 of NEST 96 Well Plate 200 µL Flat on 5,
  A5 of NEST 96 Well Plate 200 µL Flat on 5,
  A6 of NEST 96 Well Plate 200 µL Flat on 5,
  A7 of NEST 96 Well Plate 200 µL Flat on 5,
  A8 of NEST 96 Well Plate 200 µL Flat on 5,
  A9 of NEST 96 Well Plate 200 µL Flat on 5,
  A10 of NEST 96 Well Plate 200 µL Flat on 5,
  A11 of NEST 96 Well Plate 200 µL Flat on 5,
  A12 of NEST 96 Well Plate 200 µL Flat on 5],
 'B': [B1 of NEST 96 Well Plate 200 µL Flat on 5,
  B2 of NEST 96 Well Plate 200 µL Flat on 5,
  B3 of NEST 96 Well Plate 200 µL Flat on 5,
  B4 of NEST 96 Well Plate 200 µL Flat on 5,
  B5 of NEST 96 Well Plate 200 µL Flat on 5,
  B6 of NEST 96 Well Plate 200 µL Flat on 5,
  B7 of NEST 96 Well Plate 200 µL Flat on 5,
  B8 of NEST 96 Well Plate 200 µL Flat on 5,
  B9 of NEST 96 Well Plate 200 µL Flat on 5,
  B10 of NEST 96 Well Plate 200 µL Flat o

In [43]:
def get_rows(column, start_row_index, end_row_index):
    wells = []
    for i in range(start_row_index, end_row_index + 1):
        ascii_num = i + 64
        wells.append(chr(ascii_num)+str(column))
    return wells

In [46]:
get_rows(3,1,1)

['A3']

In [1]:
def get_wells_for_digit(digit, column_offset):
    wells = []
    match digit:
        case 0:
            wells + get_rows(column_offset, 1, 8)
            wells + get_rows(column_offset + 1, 1, 1)
            wells + get_rows(column_offset + 1, 8, 8)
            wells.append(get_rows(column_offset + 2, 1, 8))
            return wells
        case 1:
            wells + get_rows(column_offset + 1, 1, 8)
            wells + get_rows(column_offset, 2, 2)
            wells + get_rows(column_offset, 8, 8)
            wells + get_rows(column_offset + 2, 1, 1)
            return wells
        case 2:
            wells + get_rows(column_offset, 1, 1)
            wells + get_rows(column_offset + 1, 1, 1)
            wells + get_rows(column_offset + 2, 1, 5)

            wells + get_rows(column_offset + 1, 5, 5)
            wells + get_rows(column_offset, 5, 8)

            wells + get_rows(column_offset + 1, 8, 8)
            wells + get_rows(column_offset + 2, 8, 8)
            return wells
        case 3:
            wells + get_rows(column_offset + 2, 1, 8)

            wells + get_rows(column_offset, 1, 1)
            wells + get_rows(column_offset + 1, 1, 1)

            wells + get_rows(column_offset, 5, 5)
            wells + get_rows(column_offset + 1, 5, 5)

            wells + get_rows(column_offset, 8, 8)
            wells + get_rows(column_offset + 1, 8, 8)
            return wells
        case 4:
            wells + get_rows(column_offset + 2, 1, 8)
            wells + get_rows(column_offset, 1, 4)
            wells + get_rows(column_offset + 1, 4, 4)

            return wells
        case 5:
            wells + get_rows(column_offset, 1, 4)
            wells + get_rows(column_offset + 2, 4, 8)

            wells + get_rows(column_offset + 1, 1, 1)
            wells + get_rows(column_offset + 2, 1, 1)

            wells + get_rows(column_offset + 1, 4, 4)

            wells + get_rows(column_offset, 8, 8)
            wells + get_rows(column_offset, 8, 8)
            return wells
        case 6:
            wells + get_rows(column_offset, 1, 8)
            wells + get_rows(column_offset + 2, 5, 8)

            wells + get_rows(column_offset + 1, 1, 1)
            wells + get_rows(column_offset + 2, 1, 1)
            wells + get_rows(column_offset + 1, 5, 5)
            wells + get_rows(column_offset + 1, 8, 8)

            return wells
        case 7:
            wells + get_rows(column_offset + 2, 1, 8)

            wells + get_rows(column_offset, 1, 1)
            wells + get_rows(column_offset + 1, 1, 1)

            return wells
        case 8:
            wells + get_rows(column_offset, 1, 8)
            wells + get_rows(column_offset + 2, 1, 8)

            wells + get_rows(column_offset + 1, 1, 1)
            wells + get_rows(column_offset + 1, 4, 4)
            wells + get_rows(column_offset + 1, 8, 8)
            return wells
        case 9:
            wells + get_rows(column_offset + 2, 1, 8)
            wells + get_rows(column_offset, 1, 4)

            wells + get_rows(column_offset + 1, 1, 1)
            wells + get_rows(column_offset + 1, 4, 4)
            return wells


In [10]:
def get_wells_for_digit(digit, column_offset):
    wells = []
    match digit:
        case 0:
            wells += get_rows(column_offset, 1, 8)
            wells += get_rows(column_offset + 1, 1, 1)
            wells += get_rows(column_offset + 1, 8, 8)
            wells.append(get_rows(column_offset + 2, 1, 8))
            return wells
        case 1:
            wells += get_rows(column_offset + 1, 1, 8)
            wells += get_rows(column_offset, 2, 2)
            wells += get_rows(column_offset, 8, 8)
            wells += get_rows(column_offset + 2, 1, 1)
            return wells
        case 2:
            wells += get_rows(column_offset, 1, 1)
            wells += get_rows(column_offset + 1, 1, 1)
            wells += get_rows(column_offset + 2, 1, 5)

            wells += get_rows(column_offset + 1, 5, 5)
            wells += get_rows(column_offset, 5, 8)

            wells += get_rows(column_offset + 1, 8, 8)
            wells += get_rows(column_offset + 2, 8, 8)
            return wells
        case 3:
            wells += get_rows(column_offset + 2, 1, 8)

            wells += get_rows(column_offset, 1, 1)
            wells += get_rows(column_offset + 1, 1, 1)

            wells += get_rows(column_offset, 5, 5)
            wells += get_rows(column_offset + 1, 5, 5)

            wells += get_rows(column_offset, 8, 8)
            wells += get_rows(column_offset + 1, 8, 8)
            return wells
        case 4:
            wells += get_rows(column_offset + 2, 1, 8)
            wells += get_rows(column_offset, 1, 4)
            wells += get_rows(column_offset + 1, 4, 4)

            return wells
        case 5:
            wells += get_rows(column_offset, 1, 4)
            wells += get_rows(column_offset + 2, 4, 8)

            wells += get_rows(column_offset + 1, 1, 1)
            wells += get_rows(column_offset + 2, 1, 1)

            wells += get_rows(column_offset + 1, 4, 4)

            wells += get_rows(column_offset, 8, 8)
            wells += get_rows(column_offset, 8, 8)
            return wells
        case 6:
            wells += get_rows(column_offset, 1, 8)
            wells += get_rows(column_offset + 2, 5, 8)

            wells += get_rows(column_offset + 1, 1, 1)
            wells += get_rows(column_offset + 2, 1, 1)
            wells += get_rows(column_offset + 1, 5, 5)
            wells += get_rows(column_offset + 1, 8, 8)

            return wells
        case 7:
            wells += get_rows(column_offset + 2, 1, 8)

            wells += get_rows(column_offset, 1, 1)
            wells += get_rows(column_offset + 1, 1, 1)

            return wells
        case 8:
            wells += get_rows(column_offset, 1, 8)
            wells += get_rows(column_offset + 2, 1, 8)

            wells += get_rows(column_offset + 1, 1, 1)
            wells += get_rows(column_offset + 1, 4, 4)
            wells += get_rows(column_offset + 1, 8, 8)
            return wells
        case 9:
            wells += get_rows(column_offset + 2, 1, 8)
            wells += get_rows(column_offset, 1, 4)

            wells += get_rows(column_offset + 1, 1, 1)
            wells += get_rows(column_offset + 1, 4, 4)
            return wells

In [4]:
def get_rows(column, start_row_index, end_row_index):
    wells = []
    for i in range(start_row_index, end_row_index + 1):
        ascii_num = i + 64
        wells.append(chr(ascii_num)+str(column))
    return wells

In [11]:
get_wells_for_digit(3,1)

['A3',
 'B3',
 'C3',
 'D3',
 'E3',
 'F3',
 'G3',
 'H3',
 'A1',
 'A2',
 'E1',
 'E2',
 'H1',
 'H2']

In [12]:
from datetime import datetime

In [16]:
now = datetime.now()
current_time = now.strftime("%H%M")
current_time

'0125'

In [17]:
palette_map = {
        'green': 'A1',
        'blue': 'A2',
        'red': 'A3',
        'yellow': 'A4'
    }

In [22]:
list(palette_map.values())

['A1', 'A2', 'A3', 'A4']

In [14]:
current_time

'01:23'

In [7]:
test = []

In [8]:
test += [3]

In [9]:
test

[3]